In [ ]:
import os
from cicliminds_lib.query.files import get_datasets
from cicliminds_lib.query.files import get_model_weights
from cicliminds.app import App
from cicliminds.settings import set_plt_reasonable_defaults
set_plt_reasonable_defaults()

In [ ]:
DATA_DIR = os.environ["DATA_DIR"]
display(DATA_DIR)
MODEL_WEIGHTS_DIR = os.environ["MODEL_WEIGHTS_DIR"]
display(MODEL_WEIGHTS_DIR)

In [ ]:
DATASETS = get_datasets(DATA_DIR);
DATASETS

In [ ]:
MODEL_WEIGHTS = get_model_weights(MODEL_WEIGHTS_DIR);
MODEL_WEIGHTS

In [ ]:
app = App(DATASETS, MODEL_WEIGHTS)
app.render()